# Option Pricing in the Multi-Period Binomial Model

<img src="binomial.png">

In [1]:
import numpy as np
import pandas as pd

In [2]:
def calibrated_parameters(n, T, r, volatility, c):
    R = np.exp(r*T/n)
    u = np.exp(volatility*np.sqrt(T/n))
    v = 1/u
    q = (np.exp((r-c)*T/n)-v)/(u-v)
    return R, u, v, q

In [3]:
def binomial_model(n, S0, u, v):
    prices = pd.DataFrame((n+1)*[(n+1)*[np.nan]])
    for i in range(n+1):
        if i==0:
            prices.iloc[-1,0] = S0
        else:
            for j in range(i):
                prices.iloc[-j-1,i] = v*prices.iloc[-j-1,i-1]
                prices.iloc[-j-2,i] = u*prices.iloc[-j-1,i-1]
    return prices.fillna('')

def european_pricing_model(prices, payoff, q, R):
    l = prices.shape[0]
    option_prices = pd.DataFrame(l*[l*[np.nan]])
    for i in range(l):
        if i==0:
            option_prices.iloc[:,-1] = payoff(prices.iloc[:,-1])
        else:
            for j in range(l-i):
                E = q*option_prices.iloc[-j-2,-i] + (1-q)*option_prices.iloc[-j-1,-i]
                option_prices.iloc[-j-1,-i-1] = (1/R)*E
    return option_prices.fillna('')

def american_pricing_model(prices, payoff, q, R):
    exercised_period = prices.shape[0]-1
    l = prices.shape[0]
    option_prices = pd.DataFrame(l*[l*[np.nan]])
    for i in range(l):
        if i==0:
            option_prices.iloc[:,-1] = payoff(prices.iloc[:,-1])
        else:
            for j in range(l-i):
                E = q*option_prices.iloc[-j-2,-i] + (1-q)*option_prices.iloc[-j-1,-i]                
                option_prices.iloc[-j-1,-i-1] = np.maximum(payoff(prices.iloc[-j-1,-i-1]), (1/R)*E)
                if option_prices.iloc[-j-1,-i-1]!=(1/R)*E:
                    exercised_period = l-i-1
    return option_prices.fillna(''), exercised_period

def call_payoff(K):
    def f(S):
        return np.maximum(S-K,0)
    return f

def put_payoff(K):
    def f(S):
        return np.maximum(K-S,0)
    return f

-----

In [4]:
n = 15
T = 0.25
S0 = 100
r = 2./100
volatility = 30./100
K = 110
c = 1./100

R, u, v, q = calibrated_parameters(n=n, 
                                   T=T, 
                                   r=r, 
                                   volatility=volatility, 
                                   c=c)

In [5]:
binomial_prices = binomial_model(n=n, 
                                 S0=S0, 
                                 u=u, 
                                 v=v)
binomial_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,178.773151
1,,,,,,,,,,,,,,,171.981662,165.448178
2,,,,,,,,,,,,,,165.448178,159.162897,153.116390
3,,,,,,,,,,,,,159.162897,153.11639,147.299587,141.703761
4,,,,,,,,,,,,153.11639,147.299587,141.703761,136.320517,131.141779
5,,,,,,,,,,,147.299587,141.703761,136.320517,131.141779,126.159778,121.367041
6,,,,,,,,,,141.703761,136.320517,131.141779,126.159778,121.367041,116.756377,112.320870
7,,,,,,,,,136.320517,131.141779,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961
8,,,,,,,,131.141779,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961,100.0,96.201058
9,,,,,,,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961,100.0,96.201058,92.546435,89.030649


-----

In [6]:
european_call_prices = european_pricing_model(prices=binomial_prices, 
                                              payoff=call_payoff(K), 
                                              q=q, 
                                              R=R)
european_call_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,68.773151
1,,,,,,,,,,,,,,,61.989661,55.448178
2,,,,,,,,,,,,,,55.466347,49.173033,43.116390
3,,,,,,,,,,,,,49.193281,43.138669,37.3117,31.703761
4,,,,,,,,,,,,43.160916,37.335901,31.729843,26.334459,21.141779
5,,,,,,,,,,,37.360069,31.755892,26.362319,21.171381,16.175414,11.367041
6,,,,,,,,,,31.781908,26.390144,21.200949,16.206659,11.399901,6.77358,2.320870
7,,,,,,,,,26.470302,21.333695,16.4413,11.833683,7.598238,3.914373,1.142578,0.000000
8,,,,,,,,21.52681,16.744146,12.301781,8.293175,4.863172,2.212458,0.562498,0.0,0.000000
9,,,,,,,17.072246,12.761076,8.904579,5.614031,3.018073,1.229705,0.276921,0.0,0.0,0.000000


In [7]:
american_call_prices, exercised_period = american_pricing_model(prices=binomial_prices, 
                                                                payoff=call_payoff(K), 
                                                                q=q, 
                                                                R=R)
american_call_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,68.773151
1,,,,,,,,,,,,,,,61.989661,55.448178
2,,,,,,,,,,,,,,55.466347,49.173033,43.116390
3,,,,,,,,,,,,,49.193281,43.138669,37.3117,31.703761
4,,,,,,,,,,,,43.160916,37.335901,31.729843,26.334459,21.141779
5,,,,,,,,,,,37.360069,31.755892,26.362319,21.171381,16.175414,11.367041
6,,,,,,,,,,31.781908,26.390144,21.200949,16.206659,11.399901,6.77358,2.320870
7,,,,,,,,,26.470302,21.333695,16.4413,11.833683,7.598238,3.914373,1.142578,0.000000
8,,,,,,,,21.52681,16.744146,12.301781,8.293175,4.863172,2.212458,0.562498,0.0,0.000000
9,,,,,,,17.072246,12.761076,8.904579,5.614031,3.018073,1.229705,0.276921,0.0,0.0,0.000000


In [8]:
exercised_period

15

-----

In [9]:
european_put_prices = european_pricing_model(prices=binomial_prices, 
                                             payoff=put_payoff(K), 
                                             q=q, 
                                             R=R)
european_put_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,0.000000
1,,,,,,,,,,,,,,,0.0,0.000000
2,,,,,,,,,,,,,,0.0,0.0,0.000000
3,,,,,,,,,,,,,0.0,0.0,0.0,0.000000
4,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.000000
5,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.000000
6,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,,,,,,,,,0.052366,0.103212,0.20343,0.400957,0.790279,1.557628,3.07006,6.051039
8,,,,,,,,0.266827,0.475101,0.836266,1.450873,2.470589,4.102662,6.574872,9.980005,13.798942
9,,,,,,,0.77206,1.262808,2.02797,3.185646,4.871035,7.203452,10.216964,13.757695,17.432328,20.969351


In [10]:
american_put_prices, exercised_period = american_pricing_model(prices=binomial_prices, 
                                                               payoff=put_payoff(K), 
                                                               q=q, 
                                                               R=R)
american_put_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,0.000000
1,,,,,,,,,,,,,,,0.0,0.000000
2,,,,,,,,,,,,,,0.0,0.0,0.000000
3,,,,,,,,,,,,,0.0,0.0,0.0,0.000000
4,,,,,,,,,,,,0.0,0.0,0.0,0.0,0.000000
5,,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.000000
6,,,,,,,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,,,,,,,,,0.052366,0.103212,0.20343,0.400957,0.790279,1.557628,3.07006,6.051039
8,,,,,,,,0.267,0.475442,0.836938,1.452198,2.4732,4.107809,6.585017,10.0,13.798942
9,,,,,,,0.773013,1.264518,2.031012,3.190988,4.880278,7.219137,10.242885,13.798942,17.453565,20.969351


In [11]:
exercised_period

5